# Part II: Model Development

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
import joblib

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


## Reload the smart sample here

In [2]:
# Reload your smart sampling from local file 
# ----------------------------------
X, y, train_undersamp = joblib.load('data/sample-data-v1.pkl')


In [3]:
# Subset easier to manage size for testing pipelines
train_undersamp_less = train_undersamp.groupby('went_on_backorder').apply(lambda x: x.sample(frac=0.1))
train_undersamp_less = pd.DataFrame(train_undersamp_less)
# Split back into X and y
X_less = train_undersamp_less.iloc[:, :-1]
y_less = train_undersamp_less.went_on_backorder


## Normalize/standardize the data if required; otherwise ignore. You can perform this step inside the pipeline (if required). 

In [4]:
'''# Standardize data
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
# Combine X_scaled and y
train_under_stand = pd.DataFrame()
train_under_stand = X_scaled
train_under_stand.reset_index(drop = True)
train_under_stand['went_on_backorder'] = y

print("Mean of the dataset features")
print(scaler.mean_)
print("Variance of data")
print(scaler.scale_)
print("-" * 35)

print("# Scaled data:")
print(X_scaled)

print("# Mean of scaled data")
print(X_scaled.mean(axis = 0))
print("# Variance of scaled data")
print(X_scaled.std(axis = 0))'''


'# Standardize data\nscaler = preprocessing.StandardScaler().fit(X)\nX_scaled = scaler.transform(X)\n# Combine X_scaled and y\ntrain_under_stand = pd.DataFrame()\ntrain_under_stand = X_scaled\ntrain_under_stand.reset_index(drop = True)\ntrain_under_stand[\'went_on_backorder\'] = y\n\nprint("Mean of the dataset features")\nprint(scaler.mean_)\nprint("Variance of data")\nprint(scaler.scale_)\nprint("-" * 35)\n\nprint("# Scaled data:")\nprint(X_scaled)\n\nprint("# Mean of scaled data")\nprint(X_scaled.mean(axis = 0))\nprint("# Variance of scaled data")\nprint(X_scaled.std(axis = 0))'

In [5]:
# Normalize data
train_undersamp_normalized = preprocessing.normalize(train_undersamp, axis = 0, norm = 'l2')
# Split into X and y
train_undersamp_norm = pd.DataFrame(train_undersamp_normalized)
train_undersamp_norm.columns=train_undersamp.columns
X_norm = train_undersamp_norm.iloc[:, :-1]
y_norm = train_undersamp_norm.went_on_backorder

print('# Scaled & normalized values:')
print(train_undersamp_normalized)

print("# All have unit norm")
print(np.linalg.norm(train_undersamp_normalized, axis = 0))

#train_undersamp_norm.info()

# Scaled & normalized values:
[[1.97363517e-05 1.52438006e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 0.00000000e+00]
 [3.28939195e-05 9.14628036e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 0.00000000e+00]
 [3.28939195e-05 6.09752024e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 0.00000000e+00]
 ...
 [1.31575678e-05 6.09752024e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 9.59456104e-03]
 [2.96045275e-05 9.14628036e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 9.59456104e-03]
 [6.57878390e-06 6.85971027e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 9.59456104e-03]]
# All have unit norm
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
'''# Normalize data
X_norm =  = preprocessing.normalize(X, axis = 0, norm = 'l2')

print('# Scaled & normalized values:')
print(X_norm)

print("# All have unit norm")
print(np.linalg.norm(X_norm, axis = 0))

X_norm.info()'''

'# Normalize data\nX_norm =  = preprocessing.normalize(X, axis = 0, norm = \'l2\')\n\nprint(\'# Scaled & normalized values:\')\nprint(X_norm)\n\nprint("# All have unit norm")\nprint(np.linalg.norm(X_norm, axis = 0))\n\nX_norm.info()'

## Split the data into Train/Test

In [7]:
# Split original dataset into training and testing sets
# Small sample to test pipelines
X_train, X_test, y_train, y_test = train_test_split(X_less, y_less, test_size = 0.2)
'''# Full sample
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)'''
'''# Split normalized dataset into training and testing sets
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size = 0.2)'''


'# Split normalized dataset into training and testing sets\nX_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size = 0.2)'

## Developing Pipeline

In this section, we design an operationalized machine learning pipeline, which includes:

* Anomaly detection
* Dimensionality Reduction
* Train a classification model


We are free to use any of the models that we learned in the past or we can use new models. Here is a pool of methods: 

### Pool of Anomaly Detection Methods (Discussed in M4)
1. IsolationForest
2. EllipticEnvelope
3. LocalOutlierFactor
4. OneClassSVM
5. SGDOneClassSVM

### Pool of Feature Selection Methods (Discussed in M3)

1. VarianceThreshold
1. SelectKBest with any scoring method (e.g, chi, f_classif, mutual_info_classif)
1. SelectKPercentile
3. SelectFpr, SelectFdr, or  SelectFwe
1. GenericUnivariateSelect
2. PCA
3. Factor Analysis
4. Variance Threshold
5. RFE
7. SelectFromModel


### Classification Methods (Discussed in M1-M2
1. Decision Tree
2. Random Forest
3. Logistic Regression
4. Naive Bayes
5. Linear SVC
6. SVC with kernels
7. KNeighborsClassifier
8. GradientBoostingClassifier
9. XGBClassifier
10. LGBM Classifier



It is difficult to fit an anomaly detection method in the sklearn pipeline without writing custom codes. For simplicity, we avoid fitting an anomaly detection method within a pipeline. So we can create the workflow in two steps. 
* Step I: fit an outlier with the training set
* Step II: define a pipeline using a feature selection and a classification method. Then cross-validate this pipeline using the training data without outliers. 
* Note: if your smart sample is somewhat imbalanced, you might want to change the scoring method in GridSearchCV (see the [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).


Once we fit the pipeline with gridsearch, we identify the best model and give an unbiased evaluation using the test set that we created in Part II. For unbiased evaluation we report confusion matrix, precision, recall, f1-score, accuracy, and other measures if you like. 

**Optional: Those who are interested in writing custom codes for adding an outlier detection method into the sklearn pipeline, please follow this discussion [thread](https://stackoverflow.com/questions/52346725/can-i-add-outlier-detection-and-removal-to-scikit-learn-pipeline).**


**Note:** <span style='background:yellow'>We will be using Grid Search to find the optimal parameters of the pipelines.</span>

You can add more notebook cells or import any Python modules as needed.

In [8]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.feature_selection import (SelectKBest, VarianceThreshold, 
                                       SelectPercentile, chi2, f_classif, mutual_info_classif)
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix 
from numpy import where
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline

import sklearn.feature_selection


### Your 1st pipeline 
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation
  
Add cells as needed. 

In [9]:
# Add anomaly detection code  (Question #E201)
# ----------------------------------
# Construct local outlier factor
lof = LocalOutlierFactor()

# Get labels from classifier to cull outliers
lof_outliers = lof.fit_predict(X_train) == -1

X_lof = X_train[~lof_outliers]
y_lof = y_train[~lof_outliers]


In [10]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E202)
# ----------------------------------
# Define pipeline
pipe = Pipeline([
    ('PCA', PCA()),
    ('LR_model', LogisticRegression())
])

# Define parameter grid
param_grid = {'PCA__n_components': [5, 20, 30, 35, 60],
              'PCA__random_state':[17],
              'LR_model__C': [0.001, 0.1, 1.0, 10, 100], 
              'LR_model__max_iter': [1000, 2500, 5000]
              }

# Use Grid Search to train Pipeline
CV_log_reg = GridSearchCV(pipe, param_grid, n_jobs = 2, cv = 10)
CV_log_reg_model = CV_log_reg.fit(X_lof, y_lof)
#print(CV_log_reg.cv_results_)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.77882698        nan        nan        nan        nan 0.77882698
        nan        nan        nan        nan 0.77882698        nan
        nan        nan        nan 0.77882698        nan        nan
        nan        nan 0.77882698        nan        nan        nan
        nan 0.77882698        nan        nan        nan        nan
 0.77882698        nan        nan        nan        nan 0.77882698
        nan        nan        nan        nan 0.77882698        nan
        nan        nan        nan 0.77882698        nan        nan
        nan        nan 0.77882698        nan        nan        nan
        nan 0.77882698        nan        nan        nan        nan
 0.77882698        nan        nan        nan        nan 0.77882698
        nan        nan        nan        nan 0.77882698        nan
        nan        nan        nan]
  category=UserWarning

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('PCA', PCA()),
                                       ('LR_model', LogisticRegression())]),
             n_jobs=2,
             param_grid={'LR_model__C': [0.001, 0.1, 1.0, 10, 100],
                         'LR_model__max_iter': [1000, 2500, 5000],
                         'PCA__n_components': [5, 20, 30, 35, 60],
                         'PCA__random_state': [17]})

In [11]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
'''# Show parameters of trained models and their rank
pd.set_option("max_colwidth", 80)
CV_log_reg_df = pd.DataFrame(CV_log_reg.cv_results_)
print(CV_log_reg_df[['params','rank_test_score']])'''

# Evaluate best model using test data
predicted_y = CV_log_reg.predict(X_test)

'''# Show parameters of best model
best_params = CV_log_reg.best_params_
print('Best parameter:\n',best_params)'''

# Show best estimator
best_estimator = CV_log_reg.best_estimator_
print('Best estimator:\n',best_estimator) 

# Display confusion matrix
print('\nConfusion Matrix:\n',pd.DataFrame(confusion_matrix(y_test, predicted_y)))

# Create classification report
print('\nClassification Report:\n',classification_report(y_test, predicted_y))


Best estimator:
 Pipeline(steps=[('PCA', PCA(n_components=5, random_state=17)),
                ('LR_model', LogisticRegression(C=0.001, max_iter=1000))])

Confusion Matrix:
      0    1
0  137   75
1   18  205

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.65      0.75       212
           1       0.73      0.92      0.82       223

    accuracy                           0.79       435
   macro avg       0.81      0.78      0.78       435
weighted avg       0.81      0.79      0.78       435



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 2nd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [12]:
# Add anomaly detection code  (Question #E205)
# ----------------------------------
# Construct envelope
ee = EllipticEnvelope()
# Fit data to envelope
#envelope = env.fit(X_train_norm, y_train_norm)
envelope = ee.fit(X_train, y_train)

# Get labels from classifier to cull outliers
env_outliers = envelope.predict(X_train) == -1

# Re-slice X,y into a cleaned dataset with outliers excluded
X_env = X_train[~env_outliers]
y_env = y_train[~env_outliers]


/opt/conda/lib/python3.7/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [13]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E206)
# ----------------------------------
#kb = SelectKBest()
#rfc = RandomForestClassifier(random_state = 17)
#fa = FactorAnalysis()
#dtc = DecisionTreeClassifier()

# Define pipeline
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('kbest', SelectKBest(mutual_info_classif)),
        #('rfc_model', rfc)
        #('fa', FactorAnalysis()),
        ('dtc_model', DecisionTreeClassifier())
    ]
)

param_grid = [
    {
        #'fa__n_components': [5, 20, 80, 120, 480],
        'kbest__k': [3, 5, 10, 15, 20],
        'dtc_model__max_features': ['auto', 'sqrt', 'log2'],
        'dtc_model__max_leaf_nodes': [None, 15, 10, 5, 1],
        'dtc_model__random_state': [17]
    },
]

# Use Grid Search to train Pipeline
CV_dtc = GridSearchCV(pipe, param_grid = param_grid, n_jobs = 2, cv = 10)
CV_dtc_model = CV_dtc.fit(X_env, y_env)
#print(CV_dtc.cv_results_)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.81768741 0.80487098 0.79594561 0.77802139        nan 0.75685938
 0.81196717 0.80681447 0.81189776        nan 0.7453985  0.80487098
 0.80042055 0.81450269        nan 0.73195329 0.80233954 0.79084599
 0.80552017        nan        nan        nan        nan        nan
        nan 0.82023518 0.81704638 0.79848114 0.78440307        nan
 0.75685938 0.82028417 0.82024743 0.81769557        nan 0.74091132
 0.80105341 0.80234771 0.80105341        nan 0.73195329 0.80554467
 0.77804589 0.80423812        nan        nan        nan        nan
        nan        nan 0.81703005 0.80873755 0.79657031 0.7792667
        nan 0.75494855 0.82597175 0.81450269 0.80933366        nan
 0.73898824 0.79723992 0.80170668 0.80682672        nan 0.73195329
 0.80425037 0.78574637 0.8061612         nan        nan        nan
        nan        nan        nan]
  category=UserWarning


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('kbest',
                                        SelectKBest(score_func=<function mutual_info_classif at 0x7f67fcd56730>)),
                                       ('dtc_model',
                                        DecisionTreeClassifier())]),
             n_jobs=2,
             param_grid=[{'dtc_model__max_features': ['auto', 'sqrt', 'log2'],
                          'dtc_model__max_leaf_nodes': [None, 15, 10, 5, 1],
                          'dtc_model__random_state': [17],
                          'kbest__k': [3, 5, 10, 15, 20]}])

In [14]:
# Given an unbiased evaluation  (Question #E207)
# ----------------------------------
'''# Show parameters of trained models and their rank
pd.set_option("max_colwidth", 80)
CV_dtc_df = pd.DataFrame(CV_dtc.cv_results_)
print(CV_dtc_df[['params','rank_test_score']])'''

# Evaluate best model using test data
predicted_y = CV_dtc.predict(X_test)

'''# Show parameters of best model
best_params = CV_dtc.best_params_
print('Best parameter:\n',best_params)'''

# Show best estimator
best_estimator = CV_dtc.best_estimator_
print('Best estimator:\n',best_estimator) 

# Display confusion matrix
print('\nConfusion Matrix:\n',pd.DataFrame(confusion_matrix(y_test, predicted_y)))

# Create classification report
print('\nClassification Report:\n',classification_report(y_test, predicted_y))


Best estimator:
 Pipeline(steps=[('scaler', StandardScaler()),
                ('kbest',
                 SelectKBest(k=5,
                             score_func=<function mutual_info_classif at 0x7f67fcd56730>)),
                ('dtc_model',
                 DecisionTreeClassifier(max_features='log2', max_leaf_nodes=15,
                                        random_state=17))])

Confusion Matrix:
      0    1
0  155   57
1   32  191

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.73      0.78       212
           1       0.77      0.86      0.81       223

    accuracy                           0.80       435
   macro avg       0.80      0.79      0.79       435
weighted avg       0.80      0.80      0.79       435



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 3rd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [15]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------
# Construct IsolationForest
iso_forest = IsolationForest(n_estimators = 250)

iso_outliers = iso_forest.fit(X_train, y_train)

# Get labels from classifier to cull outliers
iso_outliers = iso_forest.predict(X_train) == -1

X_iso = X_train[~iso_outliers]
y_iso = y_train[~iso_outliers]


In [16]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------
# Define pipeline
pipe = Pipeline(
    [
        #('fa', FactorAnalysis()),
        #('vt', VarianceThreshold()),
        ('k_percentile', SelectPercentile()),
        ('rfc_model', RandomForestClassifier())
    ]
)

param_grid = [
    {
        #'fa__n_components': [5, 20, 80, 120, 480],
        #'fa__random_state': [17],
        #'vt__threshold': [0, 0.0001, 0.001, 0.01, 0.1],
        'k_percentile__percentile': [1, 5, 10, 50, 75],
        'rfc_model__max_features' : ['auto', 'sqrt', 'log2'],
        'rfc_model__max_depth': [None, 55, 30, 15, 1],
        'rfc_model__n_estimators':[25, 50, 100, 150, 500]
    },
]

# Use Grid Search to train Pipeline
CV_rfc = GridSearchCV(pipe, param_grid = param_grid, n_jobs = 2, cv = 10)
CV_rfc_model = CV_rfc.fit(X_iso, y_iso)
#print(CV_rfc.cv_results_)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [15] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [17]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
'''# Show parameters of trained models and their rank
pd.set_option("max_colwidth", 80)
CV_rfc_df = pd.DataFrame(CV_rfc.cv_results_)
print(CV_rfc_df[['params','rank_test_score']])'''

# Evaluate best model using test data
predicted_y = CV_rfc.predict(X_test)

'''# Show parameters of best model
best_params = CV_rfc.best_params_
print('Best parameter:\n',best_params)'''

# Show best estimator
best_estimator = CV_rfc.best_estimator_
print('\nBest estimator:\n',best_estimator) 

# Display confusion matrix
print('\nConfusion Matrix:\n',pd.DataFrame(confusion_matrix(y_test, predicted_y)))

# Create classification report
print('\nClassification Report:\n',classification_report(y_test, predicted_y))



Best estimator:
 Pipeline(steps=[('k_percentile', SelectPercentile(percentile=75)),
                ('rfc_model',
                 RandomForestClassifier(max_depth=30, n_estimators=50))])

Confusion Matrix:
      0    1
0  183   29
1   35  188

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.86      0.85       212
           1       0.87      0.84      0.85       223

    accuracy                           0.85       435
   macro avg       0.85      0.85      0.85       435
weighted avg       0.85      0.85      0.85       435



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## Compare these three pipelines and discuss your findings

## <span style="background: yellow;">Commit your code!</span> 

### Pickle the required pipeline/models for Part III.

In [18]:
# Pickle the best model
#joblib.dump(CV_log_reg_model CV_dtc_model CV_rfc_model, 'data/pipeline-v1.pkl' )


['data/pipeline-v1.pkl']

You should have made a few commits so far of this project.  
**Definitely make a commit of the notebook now!**  
Comment should be: `Final Project, Checkpoint - Pipelines done`


# Save your notebook!
## Then `File > Close and Halt`